<a href="https://colab.research.google.com/github/JScottLopez/github-ds/blob/master/TIM_6500_JSLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analyzing Conditions Related to COVID-19 Deaths

By: Joseph S. Lopez

TIM-6500 v3: Princples of Data Science

August 1, 2021

In [3]:
#import necessary Python libraries
import pandas as pd

#assigned data frame to variable
df = pd.read_csv('Conditions_Contributing_to_COVID-19_Deaths__by_State_and_Age.csv')

#used include since data set contains mixed
#df.describe(include = 'all')


In [ ]:
#determine data types for each column
df.dtypes

Data As Of             object
Start Date             object
End Date               object
Group                  object
Year                   object
Month                 float64
State                  object
Condition Group        object
Condition              object
ICD10_codes            object
Age Group              object
COVID-19 Deaths        object
Number of Mentions     object
Flag                   object
dtype: object

In [ ]:

#total = df.iloc[:, 11:12]
#print(total)

#hist = df.hist()
import numpy as np

def outliers_iqr(data):
  q1, q3 = np.percentile(data, [25, 75])
  iqr = q3 - q1
  lower_bound = q1 - (iqr * 1.5)
  upper_bound = q3 + (ieq * 1.5)
  return np. where((data > upper_bound) | (data < lower_bound))

for i in outliers_iqr(df.iloc[:, 11:12])[0]:
  df[i] = le.fit_transform(df[i].astype(str))
  print(df[i:i+1])

In [ ]:
#retrieve column counts and group by Age Group
df.groupby('Age Group').count()


,Data As Of,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,COVID-19 Deaths,Number of Mentions,Flag
Age Group,,,,,,,,,,,,,
0-24,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,23564,23624,3760
25-34,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,20881,20986,6443
35-44,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,18788,18985,8536
45-54,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,17232,17496,10092
55-64,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,17243,17526,10081
65-74,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,17904,18195,9420
75-84,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,18345,18623,8979
85+,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,18426,18723,8898
All Ages,27324,27324,27324,27324,26082,23598,27324,27324,27324,27324,21505,21778,5819


In [ ]:
#retrieve column counts and group by Condition
df.groupby('Condition').count()

,Data As Of,Start Date,End Date,Group,Year,Month,State,Condition Group,ICD10_codes,Age Group,COVID-19 Deaths,Number of Mentions,Flag
Condition,,,,,,,,,,,,,
Adult respiratory distress syndrome,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8321,8321,3559
All other conditions and causes (residual),11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8791,9412,3089
Alzheimer disease,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,9885,9888,1995
COVID-19,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,9304,9304,2576
Cardiac arrest,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8490,8491,3390
Cardiac arrhythmia,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8867,8893,3013
Cerebrovascular diseases,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8653,8691,3227
Chronic lower respiratory diseases,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8709,8746,3171
Diabetes,11880,11880,11880,11880,11340,10260,11880,11880,11880,11880,8612,8626,3268


In [46]:
#remove all non relevant values
df[df["State"].str.contains("District of Columbia|New York City|Puerto Rico|United States")==False]
df[df["Age Group"].str.contains("Not stated|All Ages")==False]
df[df["Group"].str.contains("By Year|By Total")==False]
df.head()

,Data As Of,Start Date,End Date,Group,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths,Number of Mentions,Flag
0,07/18/2021,01/01/2020,07/17/2021,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,443,462,NaN
1,07/18/2021,01/01/2020,07/17/2021,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,"1,999","2,051",NaN
2,07/18/2021,01/01/2020,07/17/2021,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,"5,293","5,434",NaN
3,07/18/2021,01/01/2020,07/17/2021,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,"15,615","16,079",NaN
4,07/18/2021,01/01/2020,07/17/2021,By Total,NaN,NaN,United States,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,"39,217","40,321",NaN


In [39]:
#drop all null values
df.dropna(subset = ["COVID-19 Deaths"], inplace=True)
print(df)

        Data As Of  Start Date  ... Number of Mentions Flag
0       07/18/2021  01/01/2020  ...                462  NaN
1       07/18/2021  01/01/2020  ...              2,051  NaN
2       07/18/2021  01/01/2020  ...              5,434  NaN
3       07/18/2021  01/01/2020  ...             16,079  NaN
4       07/18/2021  01/01/2020  ...             40,321  NaN
...            ...         ...  ...                ...  ...
273234  07/18/2021  02/01/2021  ...                132  NaN
273235  07/18/2021  03/01/2021  ...                 69  NaN
273236  07/18/2021  04/01/2021  ...                197  NaN
273237  07/18/2021  05/01/2021  ...                182  NaN
273238  07/18/2021  06/01/2021  ...                 27  NaN

[201085 rows x 14 columns]
